In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob
from pprint import pprint 

### 벡터 저장소 로드
미리 만들어둔 ChromaDB 벡터 저장소에서 임베딩된 문서들 불러옴. RAG의 핵심 재료임.
- 임베딩 모델: `text-embedding-3-small` (가성비 좋음)
- 컬렉션 이름: `chroma_test`
- 저장 경로: `./chroma_db`

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

vectorstore = Chroma(
    embedding_function=embeddings, # 어떤 임베딩 모델을 사용했는지 명시
    collection_name="chroma_test", # 저장 시 사용했던 컬렉션 이름과 동일해야 함
    persist_directory="./chroma_db", # 저장된 DB 파일들이 있는 경로
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 10


#### 벡터 저장소 로드 장단점, 팁:
- **설명**: 텍스트 문서를 숫자 벡터로 변환(임베딩)하여 저장하고, 유사도 기반으로 빠르게 검색할 수 있게 해주는 시스템임. RAG의 'Retrieval' 부분을 담당함.
- **장점**:
  - **효율적 검색**: 대량의 문서 중에서 의미적으로 유사한 내용을 빠르게 찾을 수 있음.
  - **확장성**: 문서가 늘어나도 검색 성능을 어느 정도 유지할 수 있음 (DB 종류에 따라 다름).
  - **RAG 핵심**: LLM에게 외부 지식을 제공하여 답변의 정확도와 범위를 넓히는 데 필수적임.
- **단점**:
  - **초기 비용**: 문서를 임베딩하고 저장소에 넣는 데 시간과 비용(API 사용료 등)이 발생함.
  - **저장 공간**: 임베딩 벡터와 원본 문서를 저장할 공간이 필요함.
  - **업데이트 관리**: 원본 문서가 변경되면 벡터 저장소도 주기적으로 업데이트해야 함.
- **꿀팁🍯**:
  - **임베딩 모델 선택**: `text-embedding-3-small`은 비용과 성능 사이의 균형이 좋음. 더 높은 성능을 원하면 `text-embedding-3-large` 등을 고려해볼 수 있지만 비용이 올라감.
  - `persist_directory`: 이 경로를 정확히 지정해야 저장된 DB를 제대로 불러올 수 있음. 상대 경로보다는 절대 경로를 쓰는 게 헷갈림을 줄일 수 있음.
  - **컬렉션 관리**: 하나의 DB 서비스 안에 여러 컬렉션을 만들어 용도별로 문서를 관리하면 좋음.

## 1. LCEL의 힘: 손쉽게 체인 만들기
LangChain Expression Language (LCEL)은 파이프(`|`) 연산자를 사용해 다양한 컴포넌트(프롬프트, 모델, 파서 등)를 유연하게 연결할 수 있게 해줌.

### (1) 프롬프트 + LLM: 기본 중의 기본
가장 기본적인 체인 구성. 프롬프트 템플릿을 만들고 LLM과 연결함.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.3, # 답변의 창의성 조절 (0에 가까울수록 결정적, 1에 가까울수록 창의적)
    max_tokens=100, # LLM이 생성할 최대 토큰 수 (답변 길이 제한, 비용 관리)
    )

# 프롬프트 메시지 리스트 정의 (튜플 형태로 시스템 메시지와 사용자 메시지 등을 구성)
messages = [
    ("system", "You are a helpful assistant. Answer in Korean."), # 시스템 역할 부여, LLM의 행동 방식 지시
    ("user", "{query}"), # 사용자의 실제 질문이 들어갈 자리 (템플릿 변수)
]

# 메시지 리스트로부터 ChatPromptTemplate 객체 생성
prompt = ChatPromptTemplate.from_messages(messages)

# 생성된 프롬프트 템플릿 구조 확인 (어떤 메시지들로 구성되었는지 보여줌)
print("--- 프롬프트 템플릿 구조 ---")
pprint(prompt)

--- 프롬프트 템플릿 구조 ---
ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant. Answer in Korean.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])


In [5]:
# 프롬프트 템플릿이 어떤 입력 변수를 사용하는지 확인
print(prompt.input_variables)

['query']


In [6]:
# 템플릿에 실제 값(`query`)을 넣어 프롬프트 텍스트 완성 (렌더링 과정)
prompt_text_object = prompt.invoke({"query":"테슬라 창업자는 누구인가요?"}) # invoke를 사용하면 PromptValue 객체가 나옴
print("\n--- 렌더링된 프롬프트 객체 (PromptValue) ---")
pprint(prompt_text_object)

print("\n--- 렌더링된 프롬프트 (문자열 변환) ---")
print(prompt_text_object.to_string()) # 실제 LLM에 전달될 문자열 형태


--- 렌더링된 프롬프트 객체 (PromptValue) ---
ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. Answer in Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='테슬라 창업자는 누구인가요?', additional_kwargs={}, response_metadata={})])

--- 렌더링된 프롬프트 (문자열 변환) ---
System: You are a helpful assistant. Answer in Korean.
Human: 테슬라 창업자는 누구인가요?


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 

In [7]:
# 완성된 프롬프트 텍스트(PromptValue 객체)를 LLM에 직접 입력하여 응답 받기 (LCEL 체인 사용 전)
response_from_llm_direct = llm.invoke(prompt_text_object)

# LLM 응답(AIMessage 객체)에서 내용(content)만 추출하여 출력
print("\n--- LLM 직접 호출 응답 (내용만) ---")
print(response_from_llm_direct.content)


--- LLM 직접 호출 응답 (내용만) ---
테슬라의 창업자는 엘론 머스크(Elon Musk)가 아닙니다. 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 그러나 엘론 머스크는 2004년에 테슬라에 투자하고 CEO로 취임하면서 회사의 주요 인물로 자리잡았습니다. 이후 그는 테슬라의 성장과 발전에


In [8]:
# LCEL을 사용한 체인 구성: 프롬프트와 LLM을 `|` (파이프) 연산자로 연결. 이게 핵심!
chain = prompt | llm

# 구성된 체인 정보 출력 (어떤 Runnable들이 순서대로 연결되었는지 보여줌)
print("\n--- LCEL 체인 구성 정보 ---")
pprint(chain)


--- LCEL 체인 구성 정보 ---
ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant. Answer in Korean.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023C6CF95090>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023C6CF9C810>, root_client=<openai.OpenAI object at 0x0000023C6CD19F10>, root_async_client=<openai.AsyncOpenAI object at 0x0000023C6CF95250>, model_name='gpt-4o-mini', temperature=0.3, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=100)


In [9]:
# 체인의 입력 스키마 확인 (이 체인이 어떤 입력을 받는지 JSON 스키마 형태로 보여줌)
print("\n--- 체인 입력 스키마 ---")
pprint(chain.input_schema.schema())


--- 체인 입력 스키마 ---
{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


C:\Users\ryusg\AppData\Local\Temp\ipykernel_2700\2406282562.py:3: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pprint(chain.input_schema.schema())


In [10]:
# 체인 실행 방법 1: 딕셔너리 형태로 입력 (입력 변수 이름을 키로 사용)
# 프롬프트의 input_variables가 'query'이므로, 딕셔너리 키도 'query'여야 함.
response_from_chain_dict = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인 응답(AIMessage 객체)의 내용 출력
print("\n--- 체인 실행 응답 1 (딕셔너리 입력) ---")
print(response_from_chain_dict.content)


--- 체인 실행 응답 1 (딕셔너리 입력) ---
테슬라의 창업자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타르펜닝(Mark Tarpenning) 등에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, 이후 CEO로서 회사를 이끌고 있습니다.


In [11]:
# 체인 실행 방법 2: 입력 변수가 하나일 경우, 문자열로 직접 입력 가능 (내부적으로 딕셔너리로 변환해줌)
# 이 경우, prompt.input_variables가 단 하나일 때만 가능함!
response_from_chain_str_input = chain.invoke("테슬라 창업자는 누구인가요?") # 이 response_from_chain_str_input은 아래 Output Parser에서 사용됨

# 체인 응답(AIMessage 객체)의 내용 출력
print("\n--- 체인 실행 응답 2 (문자열 직접 입력) ---")
print(response_from_chain_str_input.content)


--- 체인 실행 응답 2 (문자열 직접 입력) ---
테슬라의 창업자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 주요 인물로 자리잡았습니다.


#### 프롬프트 + LLM 체인 장단점, 팁:
- **설명**: 가장 기본적인 LLM 활용 패턴. 사용자의 질문을 받아 적절한 프롬프트를 만들고 LLM에게 전달함.
- **장점**:
  - **직관적**: 이해하고 구현하기 쉬움.
  - **기본기**: 모든 LangChain 애플리케이션의 기초가 됨.
- **단점**:
  - **출력 제어 어려움**: LLM의 응답이 AIMessage 객체로 나오므로, 후속 처리를 위해 파싱이 필요할 수 있음.
  - **환각(Hallucination)**: LLM 자체의 한계로, 사실이 아닌 내용을 생성할 수 있음 (RAG 등으로 보완 필요).
- **꿀팁🍯**:
  - **시스템 메시지 활용**: `("system", "...")` 메시지를 통해 LLM의 역할, 어투, 응답 형식 등을 구체적으로 지시하면 더 좋은 결과를 얻을 수 있음.
  - `temperature` 조절: 정보 검색처럼 정확성이 중요하면 0에 가깝게, 창의적인 글쓰기가 필요하면 0.7 이상으로 설정.
  - `max_tokens` 설정: 응답 길이를 제한하여 비용을 관리하고, 너무 긴 응답으로 인한 문제를 방지.
  - **입력 변수 확인**: `prompt.input_variables`로 프롬프트가 어떤 입력을 기다리는지 확인하고, `chain.invoke()`에 정확한 키로 값을 전달해야 함 (문자열 직접 입력은 변수가 하나일 때만!).

### (2) 출력 파서: 원하는 형태로 결과 받기
- LLM의 응답(주로 AIMessage 객체)은 바로 사용하기엔 좀 불편할 수 있음. 
- 이걸 우리가 원하는 포맷(깔끔한 문자열, JSON 딕셔너리, Pydantic 객체 등)으로 변환해주는 고마운 친구들이 바로 출력 파서임. 
- 보통 체인의 가장 마지막에 연결해서 씀.

#### a) 문자열 파싱 (StrOutputParser)
제일 흔하게 쓰는 파서. LLM 응답(AIMessage 객체)에서 실제 텍스트 내용(`content` 속성값)만 깔끔하게 뽑아줌. 복잡한 거 필요 없고 텍스트만 원할 때 딱임.

In [12]:
# 이전 셀에서 실행한 체인의 결과 (AIMessage 객체)
print("--- 파싱 전 AIMessage 객체 ---")
pprint(response_from_chain_str_input) # 위에서 `chain.invoke("테슬라 창업자는 누구인가요?")`로 얻은 AIMessage 객체임.

--- 파싱 전 AIMessage 객체 ---
AIMessage(content='테슬라의 창업자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 주요 인물로 자리잡았습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 31, 'total_tokens': 120, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BcWH43Xyv24UN1oihFAfEtg7EQxW5', 'finish_reason': 'stop', 'logprobs': None}, id='run--6735047c-9b15-4db4-83ff-c1aeccca4b03-0', usage_metadata={'input_tokens': 31, 'output_tokens': 89, 'total_tokens': 120, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

# AIMessage 객체를 StrOutputParser에 통과시키면 문자열 내용만 반환됨
parsed_string_content = output_parser.invoke(response_from_chain_str_input)
print("\n--- StrOutputParser 적용 후 (문자열) ---")
print(parsed_string_content)
print(f"타입: {type(parsed_string_content)}")


--- StrOutputParser 적용 후 (문자열) ---
테슬라의 창업자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 주요 인물로 자리잡았습니다.
타입: <class 'str'>


In [14]:
# 체인에 StrOutputParser 연결: prompt | llm | output_parser
str_chain = prompt | llm  | StrOutputParser() # 이렇게 바로 인스턴스를 넣어줘도 됨

query = "리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)

print("\n--- StrOutputParser 포함된 체인 실행 결과 ---")
print(str_response)
print(f"타입: {type(str_response)}") 


--- StrOutputParser 포함된 체인 실행 결과 ---
리비안(Rivian)은 2009년에 설립되었습니다.
타입: <class 'str'>


##### StrOutputParser 장단점, 팁:
- **장점**:
  - **간단명료**: 사용법이 매우 쉽고 직관적임.
  - **범용성**: LLM의 텍스트 답변을 그대로 받고 싶을 때 가장 많이 사용됨.
- **단점**:
  - **구조화 불가**: 출력에서 특정 정보만 뽑아내거나 구조화된 데이터로 만들기는 어려움.
- **꿀팁🍯**:
  - 대부분의 LLM 체인 마지막에는 `StrOutputParser()`를 붙여서 깔끔한 문자열 결과를 얻는 경우가 많음.
  - 챗봇처럼 사용자에게 직접 보여줄 텍스트를 생성할 때 유용함.

#### b) JSON 출력 (JsonOutputParser)
- LLM이 JSON 형식의 문자열을 뱉어내면, 이걸 진짜 파이썬 딕셔너리나 리스트로 변환해줌. 
- 이걸 쓰려면 LLM에게 프롬프트로 "결과를 JSON 형식으로 줘!"라고 확실하게 요청해야 함.

In [15]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

# 기본 체인 (prompt | llm)을 사용. LLM에게 JSON 형식으로 출력하라고 요청함.
# 주의: LLM이 항상 완벽한 JSON을 주진 않을 수 있음!
json_response_from_llm = chain.invoke("테슬라 창업자에 대한 정보를 name과 title 필드를 포함한 JSON 형식으로 알려줘.") 
print("--- LLM의 JSON 형식 응답 (AIMessage) ---")
pprint(json_response_from_llm) # AIMessage 객체, content 안에 JSON "문자열"이 들어있음

--- LLM의 JSON 형식 응답 (AIMessage) ---
AIMessage(content='다음은 테슬라 창업자에 대한 정보를 포함한 JSON 형식의 예시입니다.\n\n```json\n{\n  "name": "엘론 머스크",\n  "title": "CEO 및 제품 아키텍트"\n}\n``` \n\n이 정보는 엘론 머스크가 테슬라의 창립자이자 CEO라는 것을 반영하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 44, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BcWH8Gb7l4xqA2W0QGWdXzv8UPeoY', 'finish_reason': 'stop', 'logprobs': None}, id='run--fab61dec-902b-4ba0-8660-4588c68adc89-0', usage_metadata={'input_tokens': 44, 'output_tokens': 78, 'total_tokens': 122, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [16]:
print("\n--- LLM 응답 내용 (JSON 문자열) ---")
print(json_response_from_llm.content)


--- LLM 응답 내용 (JSON 문자열) ---
다음은 테슬라 창업자에 대한 정보를 포함한 JSON 형식의 예시입니다.

```json
{
  "name": "엘론 머스크",
  "title": "CEO 및 제품 아키텍트"
}
``` 

이 정보는 엘론 머스크가 테슬라의 창립자이자 CEO라는 것을 반영하고 있습니다.


In [17]:
# JsonOutputParser로 AIMessage의 content (JSON 문자열)를 파싱하여 파이썬 딕셔너리로 변환
# 만약 LLM 응답이 유효한 JSON이 아니면 여기서 에러 발생 가능!
try:
    json_parser_output = json_parser.invoke(json_response_from_llm)
    print("\n--- JsonOutputParser 적용 후 (파이썬 dict) ---")
    pprint(json_parser_output)
    print(f"타입: {type(json_parser_output)}")
except Exception as e:
    print(f"\nJSON 파싱 에러: {e}")
    print("LLM이 유효한 JSON을 반환하지 않았을 수 있습니다. 프롬프트를 수정하거나 LLM의 출력을 확인하세요.")


--- JsonOutputParser 적용 후 (파이썬 dict) ---
{'name': '엘론 머스크', 'title': 'CEO 및 제품 아키텍트'}
타입: <class 'dict'>


##### JsonOutputParser 장단점, 팁:
- **장점**:
  - **구조화된 데이터**: LLM 출력을 파이썬 딕셔너리/리스트로 바로 변환해줘서 후속 데이터 처리가 매우 편해짐.
  - **API 연동 용이**: 다른 시스템과 데이터를 주고받을 때 JSON 형식이 표준처럼 쓰여서 유용함.
- **단점**:
  - **LLM 의존성**: LLM이 반드시 유효한 JSON 형식의 문자열을 생성해야 함. 가끔 형식을 안 지키거나 불완전한 JSON을 줘서 파싱 에러가 날 수 있음.
  - **프롬프트 엔지니어링**: LLM에게 JSON으로 달라고 잘 구슬려야 함. 출력 필드, 중첩 구조 등을 명확히 알려주는 게 좋음.
- **꿀팁🍯**:
  - 프롬프트에 예시 JSON 구조를 포함하거나, "반드시 JSON 형식으로만 응답해줘. 다른 설명은 붙이지 마." 같이 강력하게 지시하는 게 도움됨.
  - `try-except` 블록으로 `JsonOutputParser.invoke()` 부분을 감싸서, LLM이 이상한 값을 줬을 때의 예외 처리를 해주는 게 안전함.
  - 더 안정적인 JSON 출력을 원하면 PydanticOutputParser와 함께 LLM의 'function calling' 또는 'tool_use' 기능을 활용하는 것을 고려해볼 수 있음 (GPT 모델의 경우).

#### c) 스키마 기반 파싱 (PydanticOutputParser)
- Pydantic 모델(일종의 데이터 구조 정의 클래스)로 우리가 받고 싶은 출력물의 스키마(구조)를 미리 정의함. 
- 그리고 이 파서는 LLM에게 "이 Pydantic 모델 구조에 맞춰서 JSON으로 데이터를 줘!"라고 요청하고, 그 결과를 해당 Pydantic 모델 객체로 변환해줌.
- `get_format_instructions()` 함수로 LLM에게 어떤 형식으로 출력해야 하는지 상세한 가이드라인을 전달하는 게 핵심임. 
- 복잡한 구조의 데이터를 안정적으로 받고 싶을 때 아주 좋음.

In [18]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field # LangChain은 pydantic_v1을 내부적으로 사용함

# Pydantic 모델 정의: 원하는 출력 스키마를 클래스로 명시
# 각 필드에는 타입 힌트와 함께 Field를 사용해 설명을 추가할 수 있음. 이 설명이 LLM에게 전달됨!
class Person(BaseModel):
    """사람에 대한 정보를 담는 데이터 구조."""
    name: str = Field(..., description="그 사람의 이름") # ...는 필수 필드라는 의미
    title: str = Field(..., description="그 사람의 직함 또는 주요 역할 (예: CEO, Co-founder 등)")
    company: str = Field(..., description="그 사람이 속한 회사 또는 조직의 이름")

# PydanticOutputParser 생성 (정의한 Pydantic 모델 클래스를 인자로 전달)
person_parser = PydanticOutputParser(pydantic_object=Person)

In [19]:
print("========================================")
print("PydanticOutputParser가 생성한 프롬프트 가이드라인:")
print("----------------------------------------")
# LLM에게 어떤 형식으로 출력해야 하는지 알려주는 가이드라인(지침 문자열) 생성
format_instructions = person_parser.get_format_instructions()
print(format_instructions)
print("========================================")

PydanticOutputParser가 생성한 프롬프트 가이드라인:
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "사람에 대한 정보를 담는 데이터 구조.", "properties": {"name": {"description": "그 사람의 이름", "title": "Name", "type": "string"}, "title": {"description": "그 사람의 직함 또는 주요 역할 (예: CEO, Co-founder 등)", "title": "Title", "type": "string"}, "company": {"description": "그 사람이 속한 회사 또는 조직의 이름", "title": "Company", "type": "string"}}, "required": ["name", "title", "company"]}
```


In [20]:
# 새로운 프롬프트 템플릿 생성 (시스템 메시지에 format_instructions를 포함시켜 LLM에게 전달)
# .partial()을 사용하면 프롬프트 템플릿의 일부 변수 값을 미리 채워둘 수 있음.
pydantic_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자 질문에 답하세요. 출력은 반드시 아래 명시된 JSON 스키마를 따라야 합니다. "
            "다른 말은 하지 말고 JSON 객체만 응답하세요.\n{format_instructions}", # 여기에 Pydantic 파서가 만든 지침 삽입
        ),
        ("human", "{query}"), # 사용자 질문
    ]
).partial(format_instructions=format_instructions) # format_instructions 값을 미리 바인딩

print("\n최종 프롬프트 템플릿 (가이드라인 포함, 렌더링 예시):")
print("----------------------------------------")
rendered_pydantic_prompt = pydantic_prompt_template.invoke({"query":"테슬라 창업자는 누구인가요?"})
print(rendered_pydantic_prompt.to_string())
print("========================================")


최종 프롬프트 템플릿 (가이드라인 포함, 렌더링 예시):
----------------------------------------
System: 사용자 질문에 답하세요. 출력은 반드시 아래 명시된 JSON 스키마를 따라야 합니다. 다른 말은 하지 말고 JSON 객체만 응답하세요.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "사람에 대한 정보를 담는 데이터 구조.", "properties": {"name": {"description": "그 사람의 이름", "title": "Name", "type": "string"}, "title": {"description": "그 사람의 직함 또는 주요 역할 (예: CEO, Co-founder 등)", "title": "Title", "type": "string"}, "company": {"description": "그 사람이 속한 회사 또는 조직의 이름", "title": "Company", "type": "string"}}, "required": ["name", "title", "company"]}
```
Human: 테슬라 창업자는 누구인

In [21]:
# PydanticOutputParser를 포함한 체인 구성
# pydantic_prompt_template | llm | person_parser
person_chain = pydantic_prompt_template | llm | person_parser

# 체인 실행
query_text = "일론 머스크에 대해 알려줘."
try:
    pydantic_response = person_chain.invoke({"query": query_text})
    # 체인 응답 출력 (Pydantic 모델 객체로 반환됨! 타입 체크와 자동완성 개꿀)
    print("\n--- PydanticOutputParser 포함 체인 실행 결과 (Person 객체) ---")
    pprint(pydantic_response)
    print(f"타입: {type(pydantic_response)}")
    print(f"이름: {pydantic_response.name}, 직함: {pydantic_response.title}, 회사: {pydantic_response.company}")
except Exception as e:
    print(f"\nPydantic 파싱 에러: {e}")
    print("LLM이 Pydantic 스키마에 맞는 JSON을 반환하지 않았을 수 있습니다.")


--- PydanticOutputParser 포함 체인 실행 결과 (Person 객체) ---
Person(name='일론 머스크', title='CEO, SpaceX & Tesla', company='SpaceX, Tesla, Neuralink, The Boring Company')
타입: <class '__main__.Person'>
이름: 일론 머스크, 직함: CEO, SpaceX & Tesla, 회사: SpaceX, Tesla, Neuralink, The Boring Company


##### PydanticOutputParser 장단점, 팁:
- **장점**:
  - **강력한 스키마 검증**: Pydantic 모델에 정의된 대로 타입, 필수 필드 등을 LLM 출력이 잘 따르는지 검증해줌.
  - **개발 편의성**: LLM 응답이 Pydantic 객체로 나오니, 코드에서 `.name`, `.title`처럼 속성 접근이 편하고, IDE 자동완성도 잘 됨.
  - **LLM 가이드 명확**: `get_format_instructions()`가 LLM에게 매우 구체적인 출력 형식 지침을 제공하여, LLM이 원하는 구조로 응답할 확률이 높아짐.
- **단점**:
  - **Pydantic 모델 정의 필요**: 미리 Pydantic 클래스를 만들어야 하는 약간의 수고가 있음.
  - **프롬프트 길이 증가**: `format_instructions` 내용이 프롬프트에 추가되므로 토큰 사용량이 약간 늘어날 수 있음.
  - **여전히 실패 가능성**: LLM이 지침을 100% 따른다는 보장은 없으므로, 복잡한 스키마에서는 여전히 파싱 에러가 발생할 수 있음 (JsonOutputParser보다는 안정적임).
- **꿀팁🍯**:
  - Pydantic 모델의 `Field`에 `description`을 자세히 써주면 LLM이 각 필드의 의미를 더 잘 이해하고 적절한 값을 채워줌.
  - 시스템 프롬프트에 "다른 말은 절대 하지 말고, 요청한 JSON 객체만 생성해줘." 같이 아주 단호하게 명령하면 LLM이 이상한 사족을 붙이는 걸 줄일 수 있음.
  - LLM 모델이 function calling/tool use를 지원한다면, Pydantic 모델을 그 기능과 결합하여 더 안정적으로 구조화된 출력을 얻을 수 있음. LangChain은 이를 위한 통합도 제공함 (`ChatOpenAI.with_structured_output(Person)`).

## 2. LLM 호출, 다양하게 활용하기
LLM 객체는 단순히 `invoke`로 한 번 호출하고 끝나는 게 아님. `stream`으로 실시간 응답을 받거나, `batch`로 여러 질문을 한 방에 처리하는 등 유용한 기능들이 많음.

### (1) stream: 실시간 응답 스트리밍
- LLM 답변이 길 경우, 한 번에 다 나올 때까지 기다리면 사용자는 지루함. 
- `stream`을 쓰면 LLM이 생성하는 대로 토큰(단어 조각) 단위로 바로바로 받아볼 수 있음. 
- 사용자 경험(UX) 향상에 아주 좋음. 터미널에서 볼 땐 `flush=True` 옵션을 써야 글자가 바로바로 찍힘.

In [22]:
import time 

print("스트리밍 응답 시작 (LLM 직접 호출):")
full_response = ""
for chunk in llm.stream("테슬라의 역사에 대해 자세히 설명해줘. 한국어로."): # 체인이 아닌 llm 객체 자체의 stream 사용
    # chunk는 AIMessageChunk 객체임. content 속성에 생성된 토큰(텍스트 조각)이 들어있음
    print(chunk.content, end="", flush=True) # end=""는 줄바꿈 방지, flush=True는 버퍼 비우고 즉시 출력
    full_response += chunk.content
    time.sleep(0.05) # 너무 빠르면 눈으로 보기 힘드니 약간의 딜레이 (실제 서비스에선 불필요)
print("\n--- 스트리밍 응답 종료 ---")
print(f"\n--- 전체 응답 내용 ---\n{full_response}")

스트리밍 응답 시작 (LLM 직접 호출):
테슬라(Tesla, Inc.)는 전기차 및 에너지 저장 솔루션을 개발하는 미국의 기업으로, 2003년에 설립되었습니다. 회사의 이름은 전기공학의 선구자인 니콜라 테슬라(Nikola Tesla)에서 유래하였습니다. 테슬라의 역사와 주요 이정표를 살펴보면 다음과 같습니다.

### 설립과 초기 단계 (2003-2008)
- **200
--- 스트리밍 응답 종료 ---

--- 전체 응답 내용 ---
테슬라(Tesla, Inc.)는 전기차 및 에너지 저장 솔루션을 개발하는 미국의 기업으로, 2003년에 설립되었습니다. 회사의 이름은 전기공학의 선구자인 니콜라 테슬라(Nikola Tesla)에서 유래하였습니다. 테슬라의 역사와 주요 이정표를 살펴보면 다음과 같습니다.

### 설립과 초기 단계 (2003-2008)
- **200


In [23]:
print("\n스트리밍 응답 시작 (체인 호출):")
# StrOutputParser가 포함된 체인도 스트리밍 가능. 이때는 문자열 청크가 나옴.
str_chain_for_stream = prompt | llm | StrOutputParser()
full_response_from_chain = ""
for chunk_str in str_chain_for_stream.stream({"query": "리비안 R1T의 주요 특징은 뭐야? 한국어로 알려줘."}):
    print(chunk_str, end="", flush=True)
    full_response_from_chain += chunk_str
print("\n--- 스트리밍 응답 종료 (체인) ---")
print(f"\n--- 전체 응답 내용 (체인) ---\n{full_response_from_chain}")


스트리밍 응답 시작 (체인 호출):
리비안 R1T는 전기 픽업 트럭으로, 여러 가지 주요 특징을 가지고 있습니다. 다음은 그 주요 특징들입니다:

1. **전기 파워트레인**: R1T는 전기 모터를 사용하여 뛰어난 가속력과 성능을 제공합니다. 다양한 배터리 옵션이 있어 주행 거리도 선택할 수 있습니다.

2. **4륜 구동 시스템**: 모든 바퀴에 전력을
--- 스트리밍 응답 종료 (체인) ---

--- 전체 응답 내용 (체인) ---
리비안 R1T는 전기 픽업 트럭으로, 여러 가지 주요 특징을 가지고 있습니다. 다음은 그 주요 특징들입니다:

1. **전기 파워트레인**: R1T는 전기 모터를 사용하여 뛰어난 가속력과 성능을 제공합니다. 다양한 배터리 옵션이 있어 주행 거리도 선택할 수 있습니다.

2. **4륜 구동 시스템**: 모든 바퀴에 전력을


### (2) batch: 여러 질문 한 번에 처리
- 질문 여러 개를 리스트로 묶어서 LLM에게 한 방에 보내고, 답변도 리스트로 한 번에 받을 수 있음. 
- API 호출을 한 번으로 줄여서 네트워크 오버헤드를 줄이고, 경우에 따라 더 효율적으로 요청을 관리할 수 있음 (특히 LLM 프로바이더가 배치 처리에 최적화되어 있다면).

In [24]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?",
    "현대자동차의 전기차 모델은 어떤 것들이 있나요?"
]

# 여러 질문을 리스트로 전달하여 batch 처리 (LLM 객체 직접 사용)
print("--- LLM 배치 호출 시작 ---")
batch_responses_llm = llm.batch(questions)
print("--- LLM 배치 호출 완료 ---")

for i, response in enumerate(batch_responses_llm):
    print(f"\n[질문 {i+1}에 대한 LLM 응답 ({type(response)})]")
    # AIMessage 객체의 pretty_print() 메서드로 보기 좋게 출력 가능 (Jupyter 환경)
    # response.pretty_print() # 콘솔에서는 pprint가 더 나을 수 있음
    pprint(response.content)

--- LLM 배치 호출 시작 ---
--- LLM 배치 호출 완료 ---

[질문 1에 대한 LLM 응답 (<class 'langchain_core.messages.ai.AIMessage'>)]
('테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Marc '
 'Tarpenning), 제프 스프랙(Jeffrey B. Straubel)입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 '
 '2004년에 투자자로 참여한 후 CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다.')

[질문 2에 대한 LLM 응답 (<class 'langchain_core.messages.ai.AIMessage'>)]
('리비안(Rivian)의 창업자는 RJ 스칼리(RJ Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 및 지속 '
 '가능한 운송 수단 개발에 집중하고 있습니다. 리비안은 전기 픽업트럭과 SUV 모델을 주로 생산하고 있으며, 최근에는 전기차 시장에서 '
 '주목받고 있는 기업 중 하나입니다.')

[질문 3에 대한 LLM 응답 (<class 'langchain_core.messages.ai.AIMessage'>)]
('현대자동차의 전기차 모델에는 다음과 같은 것들이 있습니다:\n'
 '\n'
 '1. **아이오닉 5 (Ioniq 5)**: 현대의 전기차 전용 플랫폼인 E-GMP를 기반으로 한 모델로, 독특한 디자인과 넓은 실내 '
 '공간, 빠른 충전 속도가 특징입니다.\n'
 '\n'
 '2. **아이오닉 6 (Ioniq 6)**: 아이오닉 5의 후속 모델로,')


In [25]:
# 체인을 사용한 배치 처리
# 체인의 입력이 딕셔너리 형태({ "query": ... })를 기대하므로, 입력 리스트도 해당 형태로 만들어야 함.
chain_inputs = [{"query": q} for q in questions]
print("\n\n--- 체인 배치 호출 시작 (str_chain 사용) ---")
# str_chain = prompt | llm | StrOutputParser()
batch_responses_chain = str_chain.batch(chain_inputs)

print("--- 체인 배치 호출 완료 ---")
for i, response_str in enumerate(batch_responses_chain):
    print(f"\n[질문 {i+1}에 대한 체인 응답 ({type(response_str)})]")
    print(response_str)



--- 체인 배치 호출 시작 (str_chain 사용) ---
--- 체인 배치 호출 완료 ---

[질문 1에 대한 체인 응답 (<class 'str'>)]
테슬라의 창업자는 엘론 머스크(Elon Musk)입니다. 하지만 테슬라의 설립 초기에는 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)도 중요한 역할을 했습니다. 엘론 머스크는 2004년에 테슬라에 투자하고 CEO로 취임하면서 회사의 성장에 큰 영향을 미쳤습니다.

[질문 2에 대한 체인 응답 (<class 'str'>)]
리비안(Rivian)의 창업자는 로버트 스카링(R.J. Scaringe)입니다. 그는 2009년에 리비안을 설립하였으며, 전기차 제조업체로서 SUV와 픽업트럭을 주로 생산하고 있습니다.

[질문 3에 대한 체인 응답 (<class 'str'>)]
현대자동차의 전기차 모델에는 다음과 같은 것들이 있습니다:

1. **아이오닉 5 (Ioniq 5)** - 현대의 전기차 전용 플랫폼인 E-GMP를 기반으로 한 모델로, 독특한 디자인과 넓은 실내 공간을 제공합니다.
2. **아이오닉 6 (Ioniq 6)** - 아이오닉 5의 후속 모델로, 세련된 외관과 높은


#### batch 호출 장단점, 팁:
- **장점**:
  - **효율성**: 여러 요청을 한 번의 API 호출로 처리하므로, 개별적으로 여러 번 호출하는 것보다 네트워크 지연 시간이나 API 호출 제한(rate limit) 관리 측면에서 유리할 수 있음.
  - **간결한 코드**: 여러 입력을 루프 돌면서 `invoke`하는 것보다 코드가 깔끔해짐.
- **단점**:
  - **동시성 한계**: 내부적으로는 순차 처리되거나 제한된 병렬성만 제공될 수 있음 (LLM 프로바이더 및 LangChain 구현에 따라 다름). 진짜 병렬 처리를 원하면 `async` 버전인 `.abatch()`와 `asyncio`를 함께 쓰는 게 좋음.
  - **전체 대기**: 모든 입력에 대한 처리가 완료될 때까지 기다려야 결과를 받을 수 있음. 일부 입력이 오래 걸리면 전체 응답이 늦어짐.
- **꿀팁🍯**:
  - 대량의 독립적인 질문(또는 입력)을 한 번에 처리해야 할 때 유용함 (예: 문서 여러 개 요약, 질문 리스트에 대한 답변 생성 등).
  - `batch` 호출 시에도 체인에 정의된 입력 형태를 따라야 함. 예를 들어 프롬프트가 `{"query": ...}` 입력을 받는다면, `batch`에도 `[{"query": "질문1"}, {"query": "질문2"}]` 와 같이 리스트 안의 각 항목을 딕셔너리로 만들어 전달해야 함.
  - 에러 핸들링: `batch` 호출 시 일부 입력에서 에러가 발생해도, 기본적으로 다른 입력들은 계속 처리하려고 시도함 (설정 가능). 결과에서 에러 여부를 확인하고 적절히 처리해야 함.

## 4. Runnable: 더 유연한 체인 구성
- LCEL의 핵심에는 `Runnable` 프로토콜(인터페이스 같은 거)이 있음. 
- 지금까지 쓴 프롬프트, LLM, 파서 등이 다 이 `Runnable`을 따르는 애들임. 
- LangChain은 이 외에도 다양한 `Runnable` 클래스들을 제공해서, 복잡한 데이터 흐름을 만들거나 파이썬 함수 같은 커스텀 로직을 체인에 쉽게 통합할 수 있게 해줌.

### (1) RunnableParallel: 병렬 실행과 데이터 매핑
- 이름처럼 여러 `Runnable`을 동시에 실행(하는 것처럼 보이게 하거나, 실제 I/O 바운드 작업은 병렬로)하거나, 입력 데이터를 딕셔너리 형태로 가공해서 다음 `Runnable`에 전달할 때 아주 유용함. 
- 특히 RAG 체인 만들 때, '문맥(context)'과 '질문(question)'을 각각 다른 경로로 준비해서 합치는 데 자주 쓰임. `operator.itemgetter`와 찰떡궁합임.

In [26]:
# RAG를 위해 벡터 저장소에서 문서를 검색하는 Retriever 준비
# Retriever도 Runnable임!
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 2}, 
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query)

print("--- 리트리버 검색 결과 (Document 객체 리스트) ---")
pprint(retrieved_docs)

# 검색된 문서(Document 객체 리스트)들의 page_content를 합쳐서 하나의 문자열로 만듦 (RAG 프롬프트에 넣기 좋게)
retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])

print("\n--- 검색된 문서 내용 (하나의 문자열로 합침) ---")
pprint(retrieved_docs_text)

--- 리트리버 검색 결과 (Document 객체 리스트) ---
[Document(id='8fe75362-a4d9-4a61-b36b-9e162828c3c8', metadata={'source': '../data\\Tesla_KR.txt'}, page_content='테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.'),
 Document(id='299aff0e-447d-44dd-af21-f37d28c0ca47', metadata={'source': '../data\\Tesla_KR.txt'}, page_content='2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.')]

--- 검색된 문서 내용 (하나의 문자열로 합침) ---
('테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.\n'
 '2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 '
 '머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 '
 '오스틴에 있음.')


In [29]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter # 딕셔너리에서 특정 키의 값을 가져올 때 아주 유용한 함수임

# RunnableParallel 구성 예시 1: 입력 딕셔너리에서 특정 키의 값을 그대로 가져와 새로운 딕셔너리 생성
# 이 setup_map은 다음 Runnable에게 {'context': ..., 'question': ...} 형태의 딕셔너리를 전달함.
setup_map_example = RunnableParallel(
    context=itemgetter("context_source") , # 입력 dict의 'context_source' 키 값을 'context' 키로 매핑
    question=itemgetter("question_source")  # 입력 dict의 'question_source' 키 값을 'question' 키로 매핑
)

# 실행: 입력으로 딕셔너리를 전달
input_dict_for_setup = {"context_source": retrieved_docs_text, "question_source": query, "extra_info": "불필요한 정보"}
runnable_parallel_output = setup_map_example.invoke(input_dict_for_setup)

print("--- RunnableParallel (itemgetter 사용) 결과 ---")
pprint(runnable_parallel_output) # 'extra_info'는 걸러지고 context와 question만 남음

--- RunnableParallel (itemgetter 사용) 결과 ---
{'context': '테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.\n'
            '2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 '
            '창립했음. 일론 머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 '
            '역할을 맡고 있음. 본사는 텍사스주 오스틴에 있음.',
 'question': '테슬라 창업자는 누구인가요?'}


##### RunnableParallel 장단점, 팁:
- **설명**: 여러 Runnable을 동시에 (또는 논리적으로 병렬인 것처럼) 실행하고, 그 결과들을 모아 딕셔너리 형태로 반환함. 각 키에 할당된 Runnable은 이전 단계의 전체 출력을 입력으로 받거나, `itemgetter` 등을 통해 특정 부분만 입력으로 받을 수 있음.
- **장점**:
  - **데이터 흐름 제어**: 체인의 다음 단계에 필요한 입력들을 명확한 딕셔너리 구조로 만들어 전달할 수 있음. (예: 프롬프트 템플릿이 `{context, question}`을 요구할 때 유용)
  - **병렬성 (잠재적)**: 각 키에 할당된 Runnable들이 서로 독립적이고 I/O 바운드 작업(네트워크 요청, 파일 읽기 등)이라면, 비동기 실행(`.ainvoke()`) 시 실제로 병렬 처리되어 성능 향상을 기대할 수 있음.
- **단점**:
  - **구조 이해**: 처음엔 데이터가 어떻게 흘러가는지, 각 Runnable이 어떤 입력을 받는지 정확히 파악해야 함.
  - **CPU 바운드 한계**: 파이썬의 GIL(Global Interpreter Lock) 때문에, CPU를 많이 쓰는 작업들은 `RunnableParallel`을 써도 완전한 병렬 실행 효과를 보기 어려움 (멀티프로세싱 필요).
- **꿀팁🍯**:
  - RAG 체인에서 질문(question)은 그대로 유지하고, 검색된 문맥(context)은 리트리버를 통해 가져와야 할 때, 이 둘을 합쳐 프롬프트에 전달하는 데 아주 유용함 (아래 RAG 예제에서 자세히 볼 거임).
  - `RunnableParallel`의 각 값으로는 다른 Runnable 뿐만 아니라, `itemgetter`, `RunnablePassthrough` (아래 설명), 심지어 고정된 값(문자열, 숫자 등)도 올 수 있음.
  - `RunnableParallel(context=retriever, question=RunnablePassthrough())` 이런 식으로 하면, 입력된 질문으로 `retriever`가 `context`를 만들고, 원본 질문은 `question` 키로 그대로 전달됨. 이것이 RAG의 단골 패턴임!

### (2) RunnablePassthrough: 입력 그대로 전달
- 받은 입력을 다음 단계로 아무런 변경 없이 그대로 넘겨주는 역할을 함. 
- 이게 왜 필요하냐면, `RunnableParallel` 안에서 여러 데이터를 조합할 때, 특정 데이터(예: 사용자의 원본 질문)는 가공하지 않고 그대로 다음 단계(예: 프롬프트)로 전달하고 싶을 때 사용함. 
- 체인 중간에 어떤 값을 추가하고 싶을 때도 씀.

In [34]:
from langchain_core.runnables import RunnablePassthrough

# RunnableParallel 내에서 RunnablePassthrough 사용 예시
# 입력이 딕셔너리 {'query': '질문 내용', 'user_id': '유저123'} 라고 가정
pass_through_example_chain = RunnableParallel(
    original_query=RunnablePassthrough(), # 입력 전체 딕셔너리를 'original_query' 키 값으로 할당
    just_the_query_text=itemgetter("query"), # 입력 딕셔너리에서 'query' 키의 값만 가져옴
) 

input_for_passthrough = {"query":"테슬라 창업자는 누구인가요?", "user_id": "guest"}
passthrough_output = pass_through_example_chain.invoke(input_for_passthrough)

print("--- RunnablePassthrough 사용 예시 결과 ---")
pprint(passthrough_output)

--- RunnablePassthrough 사용 예시 결과 ---
{'just_the_query_text': '테슬라 창업자는 누구인가요?',
 'original_query': {'query': '테슬라 창업자는 누구인가요?', 'user_id': 'guest'}}


In [35]:
# RunnablePassthrough 단독 사용 시 (입력 그대로 출력)
passthrough_alone = RunnablePassthrough()
output_alone = passthrough_alone.invoke("이것은 그대로 전달될 것입니다.")
print("\n--- RunnablePassthrough 단독 사용 결과 ---")
print(output_alone)


--- RunnablePassthrough 단독 사용 결과 ---
이것은 그대로 전달될 것입니다.


### (3) RunnableLambda: 파이썬 함수도 체인에 착!
- 직접 만든 간단한 파이썬 함수를 LCEL 체인 안에 하나의 컴포넌트처럼 끼워 넣을 수 있게 해줌. 
- 복잡한 클래스를 만들 필요 없이, 간단한 데이터 변환이나 전/후처리 로직을 체인에 넣고 싶을 때 아주 유용함. 
- 함수는 하나의 인자만 받아야 하고, 하나의 결과만 반환해야 함 (일반적으로).

In [36]:
from langchain_core.runnables import RunnableLambda

# 간단한 단어 수 세는 파이썬 함수 정의
def count_num_words(input_data):
    print(f"count_num_words 입력: {input_data}, 타입: {type(input_data)}")
    text_to_count = ""
    if isinstance(input_data, dict) and 'query_text' in input_data: # 입력이 딕셔너리이고 'query_text' 키가 있다면
        text_to_count = input_data['query_text']
    elif isinstance(input_data, str): # 입력이 문자열인 경우
        text_to_count = input_data
    return len(text_to_count.split())

# RunnableLambda로 함수 감싸기
word_counter_runnable = RunnableLambda(count_num_words)

# 테스트
print(f"단어 수 (문자열 입력): {word_counter_runnable.invoke('이것은 다섯 단어짜리 문장입니다.')}")
print(f"단어 수 (딕셔너리 입력): {word_counter_runnable.invoke({'query_text': '이것도 다섯 단어짜리 문장일까요?', 'other_key': 123})}")

count_num_words 입력: 이것은 다섯 단어짜리 문장입니다., 타입: <class 'str'>
단어 수 (문자열 입력): 4
count_num_words 입력: {'query_text': '이것도 다섯 단어짜리 문장일까요?', 'other_key': 123}, 타입: <class 'dict'>
단어 수 (딕셔너리 입력): 4


In [37]:
# RunnableParallel과 RunnableLambda 조합
# 이전 단계의 출력이 RunnablePassthrough()를 통해 그대로 count_num_words 함수의 입력으로 들어감.
# 여기서는 RunnablePassthrough()가 입력을 그대로 전달하고, 그 입력이 count_num_words로 들어감.
# count_num_words는 입력이 문자열일 것으로 기대하고 있음 (위 함수 로직상).
lambda_setup_chain = RunnableParallel(
    original_text=RunnablePassthrough(), # 입력을 'original_text' 키에 그대로 전달
    word_count=RunnablePassthrough() | word_counter_runnable, # 입력을 받아 단어 수 계산
    # 위 라인은 아래와 동일: word_count=word_counter_runnable (RunnableLambda는 이전 단계 출력을 자동으로 받음)
    # 만약 특정 키를 지정하고 싶다면 itemgetter와 조합해야 함. 
    # 예: word_count=itemgetter("some_key") | word_counter_runnable
)

test_string = "랭체인 LCEL 정말 강력하고 유용하네요."
lambda_chain_output = lambda_setup_chain.invoke(test_string)
print("\n--- RunnableLambda 조합 체인 결과 ---")
pprint(lambda_chain_output)

count_num_words 입력: 랭체인 LCEL 정말 강력하고 유용하네요., 타입: <class 'str'>

--- RunnableLambda 조합 체인 결과 ---
{'original_text': '랭체인 LCEL 정말 강력하고 유용하네요.', 'word_count': 5}


In [39]:
# 더 명확한 예시: 입력이 딕셔너리이고, 특정 키를 함수에 전달하고 싶을 때
def custom_formatter(input_dict):
    return f"질문: {input_dict['q']}, 사용자: {input_dict['u']}"

formatter_runnable = RunnableLambda(custom_formatter)

complex_input = {"q": "오늘 날씨 어때?", "u": "AI Assistant"}
formatted_string = formatter_runnable.invoke(complex_input)
print("\n--- 복잡한 입력 처리 함수 결과 ---")
print(formatted_string)


--- 복잡한 입력 처리 함수 결과 ---
질문: 오늘 날씨 어때?, 사용자: AI Assistant


##### RunnableLambda 장단점, 팁:
- **장점**:
  - **유연성/간편성**: 기존 파이썬 함수를 거의 수정 없이 바로 LCEL 체인에 통합할 수 있어서 매우 편리함.
  - **빠른 프로토타이핑**: 복잡한 Runnable 클래스를 만들지 않고도 빠르게 커스텀 로직을 테스트하고 적용 가능.
  - **재사용성**: 이미 만들어둔 유틸리티 함수들을 LCEL 체인에서 쉽게 재활용 가능.
- **단점**:
  - **함수 제약**: `RunnableLambda`에 전달되는 함수는 기본적으로 단일 인자를 받고 단일 값을 반환하는 형태로 작성하는 것이 좋음 (컨텍스트 관리나 복잡한 상태 유지는 어려움).
  - **디버깅**: 함수 내부 로직이 복잡하면, 체인 전체의 디버깅이 약간 번거로워질 수 있음 (LangSmith가 도움됨).
  - **비동기 지원**: 람다 함수가 비동기(async def)로 작성되었다면 `RunnableLambda(my_async_func)`처럼 사용하고, 체인 호출도 `.ainvoke()` 등을 사용해야 함.
- **꿀팁🍯**:
  - `RunnableLambda(lambda x: x['key'] + ' some_text')` 처럼 익명 람다 함수도 바로 사용 가능해서 코드가 더 간결해질 수 있음.
  - 리트리버가 반환한 `Document` 객체 리스트를 하나의 문자열로 합치는 `format_docs` 같은 함수를 `RunnableLambda`로 감싸서 체인에 넣으면 아주 깔끔함 (아래 RAG 예제에서 볼 거임).
  - 함수의 입출력 타입에 주의해야 함. 이전 Runnable의 출력이 현재 `RunnableLambda`에 전달되는 함수의 입력과 호환되어야 함.

## 5. 실전! RAG 파이프라인 구축
지금까지 배운 LCEL 컴포넌트들을 조합하여 질문에 대해 관련 문서를 찾아 답변하는 RAG 파이프라인을 만듦.

### (1) RAG용 프롬프트 템플릿
RAG의 핵심 프롬프트. LLM에게 '주어진 Context 안에서만 답변하고, 모르면 모른다고 해!'라고 지시하는 게 중요함. 외부 지식 사용 방지.

In [43]:
# from langchain.prompts import ChatPromptTemplate # 이미 위에서 임포트 했지만, 명시적으로 다시 보여줌

rag_template_str = """당신은 사용자의 질문에 대해 주어진 문맥(Context)만을 사용하여 답변하는 AI 어시스턴트입니다.
외부 지식이나 정보를 절대 사용하지 마세요.
만약 문맥에서 답변을 찾을 수 없다면, "주어진 정보만으로는 답변을 찾을 수 없습니다." 또는 "잘 모르겠습니다."라고만 답변하세요.
답변은 반드시 한국어로 해주세요.

[Context]
{context}

[Question] 
{question}

[Answer]
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template_str)

print("--- RAG 프롬프트 템플릿 구조 ---")
rag_prompt.pretty_print() # Jupyter에서 보기 좋게 출력하는 LangChain 방식
# pprint(rag_prompt) # pprint로도 유사하게 확인 가능

print("\n--- RAG 프롬프트 입력 변수 ---")
print(rag_prompt.input_variables)

--- RAG 프롬프트 템플릿 구조 ---
================================ Human Message =================================

당신은 사용자의 질문에 대해 주어진 문맥(Context)만을 사용하여 답변하는 AI 어시스턴트입니다.
외부 지식이나 정보를 절대 사용하지 마세요.
만약 문맥에서 답변을 찾을 수 없다면, "주어진 정보만으로는 답변을 찾을 수 없습니다." 또는 "잘 모르겠습니다."라고만 답변하세요.
답변은 반드시 한국어로 해주세요.

[Context]
{context}

[Question] 
{question}

[Answer]


--- RAG 프롬프트 입력 변수 ---
['context', 'question']


#### RAG 프롬프트 장단점, 팁:
- **장점**:
  - **환각 감소**: LLM이 아는 척하며 잘못된 정보를 말하는 것을 크게 줄여줌.
  - **최신 정보 반영**: LLM의 학습 데이터에 없는 최신 정보도 Context로 제공하면 답변에 활용 가능.
  - **특정 도메인 지식 활용**: 회사 내부 문서, 전문 자료 등 특정 분야 지식을 기반으로 답변 생성 가능.
- **단점**:
  - **Context 의존성**: 제공된 Context의 품질과 관련성이 답변의 질을 크게 좌우함. Context가 부실하면 답변도 부실해짐.
  - **Context 길이 제한**: LLM이 한 번에 처리할 수 있는 Context 길이에 제한이 있음 (모델마다 다름). 너무 긴 문서를 통째로 넣기 어려움.
- **꿀팁🍯**:
  - **'모르면 모른다고 해' 지시**: 이게 정말 중요함! 이 지시가 없으면 LLM은 어떻게든 Context를 짜깁기해서 답하려다가 이상한 말을 할 수 있음.
  - **역할 부여**: "당신은 ~ AI입니다." 처럼 LLM에게 명확한 역할을 부여하면 더 일관된 답변을 얻을 수 있음.
  - **구분자 사용**: `[Context]`, `[Question]`, `[Answer]` 처럼 명확한 구분자를 사용하면 LLM이 각 부분을 더 잘 인식함.
  - **언어 지정**: 다국어 지원 LLM이라면 답변 언어를 명시해주는 게 좋음 (예: "답변은 반드시 한국어로 해주세요.").

### (2) 리트리버 체인: 문서 가져오고 포맷팅
- RAG의 첫 번째 단계인 'Retrieval'을 담당할 체인임. 
- 사용자 질문과 가장 유사한 문서를 벡터 저장소에서 찾아오고(`retriever`), 찾은 문서들(보통 `Document` 객체 리스트로 나옴)을 LLM이 프롬프트에서 쉽게 이해하고 사용할 수 있도록 하나의 깔끔한 문자열로 합쳐주는(`format_docs` 함수) 역할을 함.

In [44]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

def format_docs(docs):
    # 각 Document 객체의 page_content (실제 텍스트 내용)를 가져와서, 두 줄의 개행문자로 구분하여 합침.
    return "\n\n".join([d.page_content for d in docs])

# 리트리버 체인 구성: 
# 1. retriever가 질문을 받아 관련 문서를 검색 (결과는 Document 객체 리스트)
# 2. 검색된 Document 리스트가 format_docs 함수(RunnableLambda로 감싸짐)로 전달되어 하나의 문자열로 변환됨
retriever_chain = retriever | RunnableLambda(format_docs)
# 위 코드는 아래와 같이 더 풀어서 쓸 수도 있음:
# retriever_runnable = RunnableLambda(retriever.invoke) # retriever의 invoke 메소드를 Runnable로 만듦
# format_docs_runnable = RunnableLambda(format_docs)
# retriever_chain = retriever_runnable | format_docs_runnable
# 하지만 retriever 자체가 이미 Runnable이므로 그냥 retriever | format_docs_runnable 로 충분함!

# 리트리버 체인 테스트
test_query_for_retriever = "테슬라 창업자는 누구인가요?"
retrieved_context_string = retriever_chain.invoke(test_query_for_retriever)

print(f"--- 리트리버 체인 실행 결과 (포맷팅된 컨텍스트 문자열) for query: '{test_query_for_retriever}' ---")
pprint(retrieved_context_string)

--- 리트리버 체인 실행 결과 (포맷팅된 컨텍스트 문자열) for query: '테슬라 창업자는 누구인가요?' ---
('테슬라(Tesla, Inc.)는 미국의 전기 자동차 및 청정 에너지 회사임.\n'
 '\n'
 '2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Marc Tarpenning)이 공동 창립했음. 일론 '
 '머스크(Elon Musk)는 초기에 주요 투자자로 참여했으며, 현재 회사의 CEO이자 제품 설계자 역할을 맡고 있음. 본사는 텍사스주 '
 '오스틴에 있음.')


### (3) RAG 체인 완성: 모든 조각 맞추기
- 지금까지 만든 조각들을 `RunnableParallel`과 파이프(`|`)를 이용해 착착 연결
1.  사용자 질문(`question`)을 받음.
2.  `RunnableParallel`을 사용해서:
    a.  한쪽에서는 위에서 만든 `retriever_chain`이 이 질문을 받아 관련 문서(`context`)를 문자열로 준비함.
    b.  다른 한쪽에서는 `RunnablePassthrough()` (또는 `itemgetter('question')` 만약 입력이 딕셔너리라면)을 사용해 원본 질문(`question`)을 그대로 가져옴.
3.  이렇게 준비된 `context`와 `question` 딕셔너리가 `rag_prompt` (RAG용 프롬프트 템플릿)로 전달되어, 최종적으로 LLM에게 보낼 프롬프트가 완성됨.
4.  이 완성된 프롬프트가 `llm` (우리가 쓸 LLM 모델)에게 전달되어 답변이 생성됨.
5.  마지막으로 `StrOutputParser()`를 사용해 LLM의 답변(AIMessage 객체)에서 깔끔한 텍스트만 추출함.

In [45]:
rag_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=500)
rag_chain = (
    RunnableParallel(
        # retriever_chain은 입력을 받아 context 문자열을 생성함.
        # RunnablePassthrough()는 입력을 그대로 question 키의 값으로 사용함.
        # 즉, 이 RunnableParallel의 입력이 '사용자 질문 문자열'이라면,
        # context에는 retriever_chain(사용자_질문_문자열)의 결과가,
        # question에는 사용자_질문_문자열 자체가 할당됨.
        context=retriever_chain,  
        question=RunnablePassthrough() 
    )
    | rag_prompt # 위에서 생성된 {'context': ..., 'question': ...} 딕셔너리를 받아 프롬프트 완성
    | rag_llm    # 완성된 프롬프트를 LLM에 전달하여 답변 생성 (결과는 AIMessage)
    | StrOutputParser() # LLM 응답(AIMessage)에서 실제 답변 문자열만 추출
)

# RAG 체인 실행
query_for_rag = "테슬라 창업자는 누구이고, 언제 창립되었나요?"
print(f"\n--- RAG 체인 실행 시작 (질문: {query_for_rag}) ---")
final_response = rag_chain.invoke(query_for_rag)
print("--- RAG 체인 실행 완료 ---")


--- RAG 체인 실행 시작 (질문: 테슬라 창업자는 누구이고, 언제 창립되었나요?) ---
--- RAG 체인 실행 완료 ---


In [46]:
# 최종 결과 출력
print("\n--- 최종 RAG 답변 ---")
pprint(final_response)

print("\n--- 다른 질문으로 테스트 --- ")
query_rivian = "리비안은 어떤 회사인가요? 주요 제품은 무엇인가요?"
final_response_rivian = rag_chain.invoke(query_rivian)
pprint(final_response_rivian)

print("\n--- 문서에 없는 내용 질문 테스트 --- ")
query_no_context = "현대자동차의 아이오닉5 배터리 용량은 얼마인가요?"
final_response_no_context = rag_chain.invoke(query_no_context)
pprint(final_response_no_context) # "잘 모르겠습니다" 류의 답변이 나와야 정상!


--- 최종 RAG 답변 ---
'테슬라의 창립자는 마틴 에버하드와 마크 타페닝이며, 2003년에 창립되었습니다.'

--- 다른 질문으로 테스트 --- 
('리비안은 미국의 전기 자동차 제조업체이자 자동차 기술 회사입니다. 주요 제품은 R1T 전기 픽업트럭과 R1S 전기 SUV로, 이들 차량은 '
 '"스케이트보드" 플랫폼을 기반으로 하며, 오프로드 성능과 장거리 주행 능력을 강조합니다.')

--- 문서에 없는 내용 질문 테스트 --- 
'주어진 정보만으로는 답변을 찾을 수 없습니다.'
